In [1]:
import os
import utilities
from maths_objects import (
    Formulas, SiteHierarchies, DataSource, FrontMatter, FormulaTable, PageTabs, 
    FormulaTableType, FormulaTableTypeSimple, FormulaTableTypeCalculus, FormulaFile)
import pandas as pd
from collections import namedtuple

data_source = DataSource()
site_hierarchy = SiteHierarchies(data_source.site_hierarchies)
formulas_by_year = Formulas(data_source.formulas_by_year)
formulas_cumulative = Formulas(data_source.formulas_by_year_cumulative)

In [3]:
###########################################
# Draft table contains proof required items
###########################################
ext_formulas = formulas_cumulative.filter_by_function(lambda x: x['Subject'] == 'Year 12 Ext 1')

calculus_table = FormulaTable(ext_formulas)
calculus_table.set_type(FormulaTableTypeSimple)

formula_columns = calculus_table.get_type().formula_columns()
formulas_in_table = []

formulas_in_table = calculus_table._to_dataframe()[formula_columns].stack()

proof_required_items = ext_formulas.proofs_required_items()

display(len(set(formulas_in_table).intersection(set(proof_required_items)))>0)


True

In [3]:
ext_formulas = formulas_cumulative.filter_by_function(lambda x: x['Subject'] == 'Year 12 Ext 1')

calculus_table = FormulaTable(ext_formulas)
calculus_table.set_type(FormulaTableTypeCalculus)

calculus_table.contains_formula_sheet_items()


True